In [ ]:
elastic_rods_dir = '../'
weaving_dir = '../../../weaving/'
import os.path as osp
import sys; sys.path.append(elastic_rods_dir); sys.path.append(weaving_dir)
import numpy as np, elastic_rods, linkage_vis
import numpy.linalg as la
from bending_validation import suppress_stdout as so
import matplotlib.pyplot as plt
from elastic_rods import EnergyType, InterleavingType

In [ ]:
# Reloadable libraries
import finite_diff, importlib
importlib.reload(finite_diff)
from finite_diff import fd_hessian_test, fd_gradient_test, gradient_convergence_plot, hessian_convergence_plot, hessian_convergence_block_plot

# weaving
import analysis_helper, ribbon_linkage_helper, mesh_vis, linkage_utils, compute_curve_from_curvature, importlib
importlib.reload(analysis_helper)
importlib.reload(ribbon_linkage_helper)
importlib.reload(mesh_vis)
importlib.reload(linkage_utils)
importlib.reload(compute_curve_from_curvature)
from analysis_helper import (compare_turning_angle,
                            is_on_sphere, 
                            get_distance_to_center_scalar_field, 
                            plot_curvatures, 
                            get_curvature_scalar_field,
                            construct_elastic_rod_loop_from_rod_segments, 
                            concatenate_rod_properties_from_rod_segments, 
                            compute_min_distance_rigid_transformation)
from ribbon_linkage_helper import (initialize_linkage, 
                                   update_rest_curvature, 
                                   set_ribbon_linkage,
                                   export_linkage_geometry_to_obj)
from compute_curve_from_curvature import (match_geo_curvature_and_edge_len, get_all_curve_pattern)
from linkage_utils import order_segments_by_ribbons, get_turning_angle_and_length_from_ordered_rods

In [ ]:
# Sphere Parameters
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
RIBBON_CS = [1, 10]
ISO_CS = [4, 4]
MODEL_PATH = osp.join(weaving_dir + 'models/equal_edge_atI.obj')
SUBDIVISION_RESOLUTION = 5

In [ ]:
# # Single Linkage Parameters
# default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206), (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),(0.1111, 0.1865, 0.5316))
# RIBBON_CS = [0.001, 0.01]
# ISO_CS = [0.01, 0.01]
# MODEL_PATH = osp.join('../../examples/' + 'single_linkage.obj')
# SUBDIVISION_RESOLUTION = 5

In [ ]:
linkage = elastic_rods.RodLinkage(MODEL_PATH, SUBDIVISION_RESOLUTION, False, InterleavingType.weaving)
driver=linkage.centralJoint()
linkage.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, RIBBON_CS, stiffAxis=elastic_rods.StiffAxis.D1))
linkage.updateSourceFrame()
linkage.set_design_parameter_config(True, True)
with so():
    elastic_rods.designParameter_solve(linkage, regularization_weight = 0.05)
    jdo = linkage.dofOffsetForJoint(driver)
    fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
    elastic_rods.compute_equilibrium(linkage, fixedVars=fixedVars)

In [ ]:
variableDP = True
gradient_convergence_plot(linkage, minStepSize=1e-12, maxStepSize=1e-1, variableDesignParameters = variableDP)

In [ ]:
hessian_convergence_plot(linkage,  minStepSize=1e-12, maxStepSize=1e-2, variableDesignParameters = True)

In [ ]:
hessian_convergence_plot(linkage,  minStepSize=1e-12, maxStepSize=1e-2, variableDesignParameters = True, hessianVectorProduct = True)

In [ ]:
grad = linkage.gradient(variableDesignParameters = variableDP)

In [ ]:
jv_offset = linkage.dofOffsetForJoint(0)
rk_offset = linkage.numDoF()
frl_offset = rk_offset + linkage.numRestKappaVars()
jrl_offset = frl_offset + linkage.numFreeRestLengths()

In [ ]:
var_types = ['FULL', 'RV', 'JV', 'RK', 'FRL', 'JRL']
var_indices = {'FULL': range(0, linkage.numExtendedDoF()),
                 'RV': range(0, jv_offset),
                'JV' : range(jv_offset, rk_offset),
                 'RK': range(rk_offset, frl_offset),
                'FRL': range(frl_offset, jrl_offset),
                'JRL': range(jrl_offset, linkage.numExtendedDoF())
              }

In [ ]:
print(var_indices)

In [ ]:
# block_error('VAR', 'RK', eps=1e-6)

In [ ]:
direction = np.zeros(linkage.numExtendedDoF())
print(linkage.numExtendedDoF())
direction[var_indices['RK']] = np.random.normal(0, 1, len(var_indices['RK']))

In [ ]:
def decompose_gradient(g):
    return {t: g[var_indices[t]] for t in var_types}

def log_hessian_error(fd_delta_gradient, analytic_delta_gradient):
    fd_components = decompose_gradient(fd_delta_gradient)
    an_components = decompose_gradient(analytic_delta_gradient)
    for k in fd_components:
        print(f'{k}\t{la.norm(fd_components[k] - an_components[k]) / la.norm(an_components[k])}')

stepSize = 1e-5
fd_delta_grad, an_delta_grad = fd_hessian_test(linkage, stepSize, etype = elastic_rods.EnergyType.Full, direction = direction, variableDesignParameters = variableDP)
log_hessian_error(fd_delta_grad, an_delta_grad)

In [ ]:
hessian_convergence_block_plot(linkage, var_types, var_indices, variableDesignParameters=variableDP)

In [ ]:
# hs = single_linkage.hessianSparsityPattern(variableDesignParameters = True)

In [ ]:
# tm = hs.getTripletMatrix()

In [ ]:
# tm.dump('test.txt')

In [ ]:
# sparsity_pattern = np.zeros((115, 115, 3))

# with open ('test.txt', 'r') as f:
#     content = f.readlines()
#     for line in content:
#         line = line.strip()
#         line = line.split('\t')
#         i = int(line[0])
#         j = int(line[1])
#         if j < 73:
#             sparsity_pattern[int(line[0])][int(line[1])] = np.array([255, 255, 255])
#         else:
#             sparsity_pattern[int(line[0])][int(line[1])] = np.array([255, 0, 0])
            
# sparsity_pattern[0][79] = np.array([0, 255, 0])
                